In [1]:
import pandas as pd
import numpy as np

In [3]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [4]:
# 타이타닉 데이터 로드 및 전처리
titanic_df = pd.read_csv("titanic_train.csv")

# 나이의 결측치를 평균값으로 대체하기
titanic_df["Age"] = titanic_df['Age'].fillna(titanic_df['Age'].mean())
titanic_df["Cabin"] = titanic_df['Cabin'].fillna("N")
titanic_df["Embarked"] = titanic_df['Embarked'].fillna("N")

# 레이블 인코딩
def encode_features(dataDF):
    features = ['Sex', 'Embarked']
    for feature in features:
        le = LabelEncoder()
        le = le.fit(dataDF[feature])
        dataDF[feature] = le.transform(dataDF[feature])
    return dataDF

titanic_df = encode_features(titanic_df)

# 불필요한 feature 제거
drop_names = ["PassengerId", "Name", "Ticket", "Cabin"]
titanic_df = titanic_df.drop(drop_names, axis=1)

# 설명 변수(X)와 타겟 변수(y) 분리
X = titanic_df.drop("Survived", axis=1)
y = titanic_df["Survived"]

# 학습 데이터와 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)

### 1. RandomForestClassifier 하이퍼파라미터 튜닝


In [5]:
# 1. 모델 설정
rf_model = RandomForestClassifier(random_state=11)

In [6]:
# 2. 그리드 서치를 위한 옵션 설정
rf_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap' : [True, False]
}

In [7]:
# 3.그리드 서치 설정
rf_grid_search = GridSearchCV(estimator=rf_model, param_grid=rf_param_grid,
                              scoring="accuracy", cv=5, n_jobs=-1)

In [8]:
# 4. 학습
rf_grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=11), n_jobs=-1,
             param_grid={'bootstrap': [True, False],
                         'max_depth': [None, 10, 20, 30],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]},
             scoring='accuracy')

In [9]:
# 5. 최적 하이퍼파라미터 및 성능 출력
print("최적의 하이퍼파라미터: ", rf_grid_search.best_params_)

최적의 하이퍼파라미터:  {'bootstrap': True, 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 100}


In [10]:
print("최고 성능: ", rf_grid_search.best_score_)

최고 성능:  0.8286811779769525


### 2. GBM 하이퍼 파라미터 튜닝

In [11]:
# 1. 모델 설정
gbm_model = GradientBoostingClassifier(random_state=11)

# 2. 하이퍼 파라미터 설정
gbm_param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.05],
    'max_depth': [3, 4, 5],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
# 3.그리드 서치 설정
gbm_grid_search = GridSearchCV(estimator=gbm_model, param_grid=gbm_param_grid,
                              scoring="accuracy", cv=5, n_jobs=-1)

# 4. 학습
gbm_grid_search.fit(X_train, y_train)

# 5. 최적 하이퍼파라미터 및 성능 출력
print("최적의 하이퍼파라미터: ", gbm_grid_search.best_params_)
print("최고 성능: ", gbm_grid_search.best_score_)

최적의 하이퍼파라미터:  {'learning_rate': 0.05, 'max_depth': 4, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 200}
최고 성능:  0.8385107849896581


In [12]:
# 그리드 서치를 이용한 최적의 파라미터를 적용한 모델
best_gbm_model = gbm_grid_search.best_estimator_

### 3. XGBoost 하이퍼 파라미터 튜닝

In [13]:
# 1. 모델 설정
xgb_model = XGBClassifier(random_state=11)

# 2. 하이퍼 파라미터 설정
xgb_param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.05],
    'max_depth': [3, 4, 5],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

# 3.그리드 서치 설정
xgb_grid_search = GridSearchCV(estimator=xgb_model, param_grid=xgb_param_grid,
                              scoring="accuracy", cv=5, n_jobs=-1)

# 4. 학습
xgb_grid_search.fit(X_train, y_train)

# 5. 최적 하이퍼파라미터 및 성능 출력
print("최적의 하이퍼파라미터: ", xgb_grid_search.best_params_)
print("최고 성능: ", xgb_grid_search.best_score_)

최적의 하이퍼파라미터:  {'colsample_bytree': 1.0, 'learning_rate': 0.05, 'max_depth': 5, 'min_child_weight': 1, 'n_estimators': 100, 'subsample': 1.0}
최고 성능:  0.83430513148823
